In [3]:
#%pip install python-dotenv
#%pip install pandas
#%pip install numpy
#%pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd 
from pathlib import Path
import numpy as np
import json
import boto3
import os
from dotenv import load_dotenv
load_dotenv()

ACCESS_KEY=os.getenv("ACESS_KEY")
SECRET_ACCESS_KEY=os.getenv("SECRET_ACCESS_KEY")
bucket_name = "credit-card-fraud-detection-system-model-parameters"
region_name = "sa-east-1"


data_path = Path('.').resolve().parent.joinpath("data", "creditcard_2023.csv")
data = pd.read_csv(data_path, dtype=str)
data.rename(columns={"id": "transaction_id"}, inplace=True)
data.drop(columns=["Amount"], inplace=True)
data["transaction_time"] = data["transaction_id"].apply(lambda x: (int(x) // 10) + 1)
for col in [f"V{i}" for i in range(1,29)]:
    data[col] = data[col].astype(float)

s3 = boto3.client('s3', region_name=region_name, aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_ACCESS_KEY)

from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.3, random_state=42)

for fraud_cls in [0, 1]:
    mu_g = train.query(f"Class == '{fraud_cls}'")[[f"V{i}" for i in range(1, 29)]].mean().to_list()
    sigma_g = np.linalg.inv(train.query(f"Class == '{fraud_cls}'")[[f"V{i}" for i in range(1, 29)]].cov())
    s3.put_object(Bucket=bucket_name, 
                  Key=f"model_parameters_{fraud_cls}.json", 
                  Body=json.dumps({"mu": mu_g, "sigma": sigma_g.tolist()}, indent=2))